### Libraries

In [1]:
import pandas as pd
import numpy as np 

import warnings
import os

from unidecode import unidecode

import wbgapi as wb

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
warnings.filterwarnings("ignore")
os.chdir("C:\\Users\\joaos\\Documents\\GitHub\\pred_se_ml\data")

### Shadow economy data from Medina, L., & Schneider, F. (2017)

In [3]:
# Importing shadow economy data from Medina, L., & Schneider, F. (2017)
se_medina = pd.read_excel("shadow_economy_medina.xlsx")
se_medina.head()

,Country,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Albania,43.18,40.18,39.45,40.07,39.18,37.07,37.59,38.16,36.04,35.3,36.04,33.67,32.64,31.72,30.89,29.58,28.53,27.12,26.91,26.10,25.41,25.52,25.68,25.78,26.21
1,Algeria,34.96,36.14,38.16,38.88,37.64,37.68,37.18,38.28,36.11,34.2,33.58,31.90,29.62,27.76,24.93,24.44,24.21,24.07,25.90,25.89,27.37,26.94,25.98,25.74,23.98
2,Angola,50.17,47.80,55.43,50.48,52.47,46.30,50.48,49.21,48.64,48.8,46.14,48.40,48.86,46.81,43.84,41.23,37.13,35.26,36.25,36.54,36.49,36.60,35.92,34.53,35.25
3,Argentina,25.22,24.41,26.59,26.22,27.18,25.32,25.20,24.00,25.83,25.4,26.94,26.19,25.37,24.32,23.21,22.63,21.93,21.87,22.97,21.64,20.80,21.62,21.57,22.02,24.99
4,Armenia,46.65,49.50,48.63,44.66,47.14,47.48,46.41,45.81,46.85,46.6,47.61,44.11,42.08,43.57,41.03,41.38,39.47,35.39,41.04,40.14,38.46,35.52,34.56,34.78,35.96


In [4]:
# Columns' names
se_medina.columns

Index(['Country ',       1991,       1992,       1993,       1994,       1995,
             1996,       1997,       1998,       1999,       2000,       2001,
             2002,       2003,       2004,       2005,       2006,       2007,
             2008,       2009,       2010,       2011,       2012,       2013,
             2014,       2015],
      dtype='object')

In [5]:
# Removing the blank space and renaming the Country column
se_medina = se_medina.rename(columns={'Country ': 'country'})

# Reshaping from wide format to long format
se_medina = se_medina.set_index(se_medina['country']).stack().reset_index(name='se_medina').rename(columns={'level_2': 'year'})
se_medina.head()

,country,level_1,se_medina
0,Albania,country,Albania
1,Albania,1991,43.18
2,Albania,1992,40.18
3,Albania,1993,39.45
4,Albania,1994,40.07


In [6]:
# Renaming the column 'level_1' to 'year'
se_medina = se_medina.rename(columns={'level_1':'year'})

# Converting the 'se_medina' column to numeric
se_medina["se_medina"] = pd.to_numeric(se_medina.se_medina, errors = 'coerce')

# Droping 'na' observations
se_medina = se_medina.dropna()

Due to different data sources, the countries names may vary from one source to another, bellow some countries' names going to be renamed for future data joining

In [7]:
se_medina['country'] = [x.replace('CentralAfricanRepublic', 'central_african_republic') for x in se_medina['country']]
se_medina['country'] = [x.replace('Congo, Dem, Rep,', 'congo_dem_rep') for x in se_medina['country']]
se_medina['country'] = [x.replace('GuineaBissau', 'guinea_bissau') for x in se_medina['country']]
se_medina['country'] = [x.replace('Hong Kong SAR, China', 'hong_kong') for x in se_medina['country']]
se_medina['country'] = [x.replace('Syrian Arab, Rep,', 'Syria') for x in se_medina['country']]
se_medina['country'] = [x.replace('Brunei Darussalam', 'brunei') for x in se_medina['country']]
se_medina['country'] = [x.replace('swaziland', 'eswatini') for x in se_medina['country']]

In [8]:
# Selecting only the first name of each country
se_medina['country'] = se_medina['country'].str.split(',').str[0].str.strip()

# Only lowercase letters
se_medina['country'] = se_medina['country'].str.lower()

# Removing accentuation
se_medina['country'] = se_medina['country'].apply(lambda x: unidecode(x))

# replacing whitespace with _ in country
se_medina['country'] = [x.replace(' ', '_') for x in se_medina['country']] 

### Collecting the dependent variables from the World Bank API

In [9]:
# List of the countries 
countries = ['ALB', 'DZA', 'AGO', 'ARG', 'ARM', 'AUS', 'AUT', 'AZE', 'BHS', 'BHR', 'BGD', 'BLR', 'BEL', 'BLZ', 'BEN', 'BTN', 'BOL', 'BIH', 'BWA', 'BRA', 'BRN',
            'BGR', 'BFA', 'BDI', 'CPV', 'KHM', 'CMR', 'CAN', 'CAF', 'TCD', 'CHL', 'CHN', 'COL', 'COM', 'COD', 'COG', 'CRI', 'CIV', 'HRV', 'CYP', 'CZE', 'DNK',
            'DOM', 'ECU', 'EGY', 'SLV', 'GNQ', 'ERI', 'EST', 'ETH', 'FJI', 'FIN', 'FRA', 'GAB', 'GMB', 'GEO', 'DEU', 'GHA', 'GRC', 'GTM', 'GIN', 'GNB', 'GUY',
            'HTI', 'HND', 'HKG', 'HUN', 'ISL', 'IND', 'IDN', 'IRN', 'IRL', 'ISR', 'ITA', 'JAM', 'JPN', 'JOR', 'KAZ', 'KEN', 'KOR', 'KWT', 'KGZ', 'LAO', 'LVA',
            'LBN', 'LSO', 'LBR', 'LBY', 'LTU', 'LUX', 'MDG', 'MWI', 'MYS', 'MDV', 'MLI', 'MLT', 'MRT', 'MUS', 'MEX', 'MDA', 'MNG', 'MAR', 'MOZ', 'MMR', 'NAM',
            'NPL', 'NLD', 'NZL', 'NIC', 'NER', 'NGA', 'NOR', 'OMN', 'PAK', 'PNG', 'PRY', 'PER', 'PHL', 'POL', 'PRT', 'QAT', 'ROU', 'RUS', 'RWA', 'SAU', 'SEN',
            'SLE', 'SGP', 'SVK', 'SVN', 'SLB', 'ZAF', 'ESP', 'LKA', 'SUR', 'SWZ', 'SWE', 'CHE', 'SYR', 'TJK', 'TZA', 'THA', 'TGO', 'TTO', 'TUN', 'TUR',
            'UGA', 'UKR', 'ARE', 'GBR', 'USA', 'URY', 'VEN', 'VNM', 'YEM', 'ZMB', 'ZWE']

In [10]:
# List of the variables
series = ['NY.GDP.DEFL.KD.ZG', # Inflation, GDP deflator (annual %)
          'SL.UEM.TOTL.NE.ZS', # Unemployment, total (% of total labor force) (national estimate)
          'NE.TRD.GNFS.ZS', # Trade (% of GDP)
          'BX.KLT.DINV.WD.GD.ZS', # Foreign direct investment, net inflows (% of GDP)
          'NE.CON.GOVT.ZS', # General government final consumption expenditure (% of GDP)
          'IC.REG.PROC', # Start-up procedures to register a business (number)
          'IC.REG.COST.PC.ZS', # Cost of business start-up procedures (% of GNI per capita)
          'IC.REG.DURS', # Time required to start a business (days)
          'IC.PRP.DURS', # Time required to register property (days)
          'IC.TAX.DURS', # Time to prepare and pay taxes (hours)
          'NY.GDP.PCAP.CD' # GDP per capita (current US$)
         ]

In [11]:
# Collecting the data
wb_variables = wb.data.DataFrame(series=series, 
                       economy=countries, 
                       time=range(1991, 2016, 1), 
                       labels=False).reset_index()

In [12]:
wb_variables.head()

,economy,series,YR1991,YR1992,YR1993,YR1994,YR1995,YR1996,YR1997,YR1998,YR1999,YR2000,YR2001,YR2002,YR2003,YR2004,YR2005,YR2006,YR2007,YR2008,YR2009,YR2010,YR2011,YR2012,YR2013,YR2014,YR2015
0,AGO,BX.KLT.DINV.WD.GD.ZS,6.388693,3.466081,4.964921,3.837037,8.529489,2.76277,5.36336,17.121191,40.167251,9.623866,24.009075,11.406192,20.081014,9.329239,-3.526657,-0.072001,-1.368762,1.896314,3.136661,-3.851112,-2.704873,-1.143768,-5.380131,2.690006,11.081339
1,AGO,IC.PRP.DURS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,335.000000,335.000000,335.000000,335.000000,335.000000,190.000000,190.000000,190.000000,190.000000,190.000000,190.000000,190.000000
2,AGO,IC.REG.COST.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1316.400000,910.000000,653.800000,498.200000,343.700000,196.800000,151.100000,226.600000,163.100000,143.100000,130.100000,118.800000,17.000000
3,AGO,IC.REG.DURS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.000000,83.000000,83.000000,83.000000,83.000000,68.000000,68.000000,66.000000,66.000000,66.000000,66.000000,66.000000,36.000000
4,AGO,IC.REG.PROC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000000,12.000000,12.000000,12.000000,12.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000


In [13]:
# Selecionando o intervalo de tempo em anos
years = []

# The range goes from 1991 to 2015
for year in range(1991, 2016, 1):
    year = "YR" + str(year)
    years.append(year)
    
# Transform from wide to long format
wb_variables = pd.melt(wb_variables, id_vars=['economy', 'series'], value_vars=years, var_name='year', value_name='value')
wb_variables = wb_variables.pivot(index=['economy', 'year'], columns='series', values='value').reset_index()

In [14]:
# Removing 'YR' in each row of the year's column
wb_variables['year'] = [x.replace('YR', '') for x in wb_variables['year']]

In [15]:
# Renaming the columns
wb_variables = wb_variables.rename(columns={'NY.GDP.DEFL.KD.ZG':'inflation',
                                        'SL.UEM.TOTL.NE.ZS':'unemployment',
                                        'NE.TRD.GNFS.ZS':'exchange',
                                        'BX.KLT.DINV.WD.GD.ZS':'fdi',
                                        'NE.CON.GOVT.ZS':'governement_spending',
                                        'IC.REG.PROC':'business_procedure',
                                        'IC.REG.COST.PC.ZS':'cost_procedures',
                                        'IC.REG.DURS':'business_time',
                                        'IC.PRP.DURS':'property_time',
                                        'IC.TAX.DURS': 'tribute_time',
                                        'NY.GDP.PCAP.CD':'gdp_pc'
                                         })

In [16]:
# Missing data by column
wb_variables.isnull().mean().sort_values(ascending=False)*100

series
tribute_time            59.057325
property_time           56.127389
cost_procedures         52.789809
business_time           52.789809
business_procedure      52.789809
unemployment            38.828025
governement_spending    11.159236
exchange                 8.840764
fdi                      2.471338
inflation                2.394904
gdp_pc                   1.324841
economy                  0.000000
year                     0.000000
dtype: float64

In [17]:
# Saving countries name
countries_name = wb.economy.DataFrame(countries).reset_index()

# Only id and name columns
countries_name = countries_name[['id', 'name']]

# Renaming the id columns
countries_name = countries_name.rename(columns={'id': 'economy'})

In [18]:
# Merging countries_name and wb_variables
wb_variables = pd.merge(wb_variables, countries_name, on='economy')

In [19]:
wb_variables.head()

,economy,year,fdi,property_time,cost_procedures,business_time,business_procedure,tribute_time,governement_spending,exchange,inflation,gdp_pc,unemployment,name
0,AGO,1991,6.388693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.309982,850.556180,NaN,Angola
1,AGO,1992,3.466081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,476.515751,657.654640,NaN,Angola
2,AGO,1993,4.964921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,917.783468,466.679163,NaN,Angola
3,AGO,1994,3.837037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2175.978955,329.691784,NaN,Angola
4,AGO,1995,8.529489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1825.495149,398.120223,NaN,Angola


In [20]:
# Just changing the columns' order
new_order = ['name', 'economy', 'year', 'fdi', 'property_time', 'cost_procedures', 'business_time', 'business_procedure', 
             'tribute_time', 'governement_spending', 'exchange', 'inflation', 'gdp_pc', 'unemployment']

wb_variables = wb_variables[new_order]

In [21]:
wb_variables.head()

,name,economy,year,fdi,property_time,cost_procedures,business_time,business_procedure,tribute_time,governement_spending,exchange,inflation,gdp_pc,unemployment
0,Angola,AGO,1991,6.388693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.309982,850.556180,NaN
1,Angola,AGO,1992,3.466081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,476.515751,657.654640,NaN
2,Angola,AGO,1993,4.964921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,917.783468,466.679163,NaN
3,Angola,AGO,1994,3.837037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2175.978955,329.691784,NaN
4,Angola,AGO,1995,8.529489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1825.495149,398.120223,NaN


### Democracy data

In [22]:
# Importing democracy data
democracy = pd.read_excel("democracy.xlsx", usecols=['country', 'year', 'democ'])
democracy.head()

,country,year,democ
0,Afghanistan,1800,1
1,Afghanistan,1801,1
2,Afghanistan,1802,1
3,Afghanistan,1803,1
4,Afghanistan,1804,1


In [23]:
# Selecting the 1991 to 2015 year range
democracy = democracy[(democracy['year'] >= 1991) & (democracy['year'] <= 2015)]
democracy.head()

,country,year,democ
191,Afghanistan,1991,0
192,Afghanistan,1992,-77
193,Afghanistan,1993,-77
194,Afghanistan,1994,-77
195,Afghanistan,1995,-77


In [24]:
# Verifying which countries are in wb_variables and democracy 
list1 = wb_variables['name'].unique()
list2 = democracy['country'].unique()

set1 = set(list1)
set2 = set(list2)

equal_elements = set1.intersection(set2)
different_elements = set1.symmetric_difference(set2)

different_elements

{'Afghanistan',
 'Bahamas, The',
 'Belize',
 'Bosnia',
 'Bosnia and Herzegovina',
 'Brunei Darussalam',
 'Cabo Verde',
 'Cape Verde',
 'Congo Brazzaville',
 'Congo Kinshasa',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 'Congo-Brazzaville',
 "Cote D'Ivoire",
 "Cote d'Ivoire",
 'Cuba',
 'Czech Republic',
 'Czechia',
 'Czechoslovakia',
 'Djibouti',
 'Egypt',
 'Egypt, Arab Rep.',
 'Eswatini',
 'Gambia',
 'Gambia, The',
 'Hong Kong SAR, China',
 'Iceland',
 'Iran',
 'Iran, Islamic Rep.',
 'Iraq',
 'Ivory Coast',
 'Korea North',
 'Korea South',
 'Korea, Rep.',
 'Kosovo',
 'Kyrgyz Republic',
 'Kyrgyzstan',
 'Lao PDR',
 'Laos',
 'Macedonia',
 'Maldives',
 'Malta',
 'Montenegro',
 'Myanmar',
 'Myanmar (Burma)',
 'Panama',
 'Russia',
 'Russian Federation',
 'Serbia',
 'Serbia and Montenegro',
 'Somalia',
 'South Sudan',
 'Sudan',
 'Sudan-North',
 'Swaziland',
 'Syria',
 'Syrian Arab Republic',
 'Taiwan',
 'Timor Leste',
 'Turkey',
 'Turkiye',
 'Turkmenistan',
 'UAE',
 'USSR',
 'United Arab Emirates',


Some of the countries shown above has different names depending on the data source, bellow these countries will be renamed to have the same name

In [25]:
democracy['country'] = [x.replace('Ivory Coast', "Cote d'Ivoire") for x in democracy['country']]
democracy['country'] = [x.replace("Cote D'Ivoire", "Cote d'Ivoire") for x in democracy['country']]
democracy['country'] = [x.replace("Congo-Brazzaville", "Congo Brazzaville") for x in democracy['country']]
democracy['country'] = [x.replace("Congo Brazzaville", "Congo, Rep.") for x in democracy['country']]
democracy['country'] = [x.replace("Congo Kinshasa", "Congo, Dem. Rep.") for x in democracy['country']]
democracy['country'] = [x.replace("Korea South", "Korea, Rep.") for x in democracy['country']]
democracy['country'] = [x.replace("Cape Verde", "Cabo Verde") for x in democracy['country']]
democracy['country'] = [x.replace("Czech Republic", "Czechia") for x in democracy['country']]
democracy['country'] = [x.replace("Egypt", "Egypt, Arab Rep.") for x in democracy['country']]
democracy['country'] = [x.replace("Swaziland", "Eswatini") for x in democracy['country']]
democracy['country'] = [x.replace("Gambia", "Gambia, The") for x in democracy['country']]
democracy['country'] = [x.replace("Iran", "Iran, Islamic Rep.") for x in democracy['country']]
democracy['country'] = [x.replace("Kyrgyzstan", "Kyrgyz Republic") for x in democracy['country']]
democracy['country'] = [x.replace("Laos", "Lao PDR") for x in democracy['country']]
democracy['country'] = [x.replace("Myanmar (Burma)", "Myanmar") for x in democracy['country']]
democracy['country'] = [x.replace("Russia", "Russian Federation") for x in democracy['country']]
democracy['country'] = [x.replace("Syria", "Syrian Arab Republic") for x in democracy['country']]
democracy['country'] = [x.replace("Turkey", "Turkiye") for x in democracy['country']]
democracy['country'] = [x.replace("UAE", "United Arab Emirates") for x in democracy['country']]
democracy['country'] = [x.replace("Venezuela", "Venezuela, RB") for x in democracy['country']]
democracy['country'] = [x.replace("Yemen", "Yemen, Rep.") for x in democracy['country']]
democracy['country'] = [x.replace("Egypt, Arab Rep.", "Egypt, Arab Rep.") for x in democracy['country']]

In [26]:
# Removing countries that are in democracy but not in variables_wb and removing other countries in specific 
countries_to_drop = ['Korea North', 'Afghanistan', 'Bosnia', 'Cuba', 'Czechoslovakia','Djibouti', 'Iraq', 'Kosovo', 'Macedonia',
                    'Maldives', 'Montenegro', 'Panama', 'Serbia and Montenegro', 'Yugoslavia', 'Somalia',
                     'South Sudan', 'Sudan', 'Sudan-North', 'Taiwan', 'Timor Leste', 'USSR', 'Uzbekistan',
                    'Serbia', 'Turkmenistan']

democracy = democracy[~democracy['country'].isin(countries_to_drop)]

In [27]:
# removing countries that are in wb_variables but not in democracy
countries_to_drop2 = ['Bahamas, The', 'Belize', 'Bosnia and Herzegovina', 'Brunei Darussalam', 'Hong Kong SAR, China',
                      'Iceland', 'Maldives', 'Malta']

wb_variables = wb_variables[~wb_variables['name'].isin(countries_to_drop2)]

In [28]:
# Renaming the column country in democracy for a merging with the wb_variables data frame
democracy = democracy.rename(columns={'country':'name'})

# Object to int
wb_variables['year'] = wb_variables['year'].astype('int64')

# Merging wb_variables and democracy
df1 = pd.merge(wb_variables, democracy, left_on=['name', 'year'], right_on=['name', 'year'])

In [29]:
df1.head()

,name,economy,year,fdi,property_time,cost_procedures,business_time,business_procedure,tribute_time,governement_spending,exchange,inflation,gdp_pc,unemployment,democ
0,Angola,AGO,1991,6.388693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.309982,850.556180,NaN,-88
1,Angola,AGO,1992,3.466081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,476.515751,657.654640,NaN,-77
2,Angola,AGO,1993,4.964921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,917.783468,466.679163,NaN,-88
3,Angola,AGO,1994,3.837037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2175.978955,329.691784,NaN,-88
4,Angola,AGO,1995,8.529489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1825.495149,398.120223,NaN,-88


In [30]:
# Renaming the classes in democ column
df1['democ'] = df1['democ'].replace(-88, 'transition')
df1['democ'] = df1['democ'].replace(-77, 'interregnum')
df1['democ'] = df1['democ'].replace(-66, 'interruption')
df1['democ'] = df1['democ'].replace(0, 'autocracy')
df1['democ'] = df1['democ'].replace(1, 'closed_anocracy')
df1['democ'] = df1['democ'].replace(2, 'closed_anocracy')
df1['democ'] = df1['democ'].replace(3, 'open_anocracy')
df1['democ'] = df1['democ'].replace(4, 'open_anocracy')
df1['democ'] = df1['democ'].replace(5, 'open_anocracy')
df1['democ'] = df1['democ'].replace(6, 'open_anocracy')
df1['democ'] = df1['democ'].replace(7, 'democracy')
df1['democ'] = df1['democ'].replace(8, 'democracy')
df1['democ'] = df1['democ'].replace(9, 'democracy')
df1['democ'] = df1['democ'].replace(10, 'full_democracy')

### Tax burden

In [31]:
# Importing the tax burden data
tax_burden = pd.read_csv('tax_burden.csv', usecols=['Name', 'Index Year', 'Tax Burden'])

# Sorting the data
tax_burden = tax_burden.sort_values(['Name', 'Index Year'], ascending=[True, True])

# Renaming the columns
tax_burden = tax_burden.rename(columns={'Name':'name', 
                         'Index Year': 'year',
                         'Tax burden': 'carga_trib'})

# Selecting the range between 1995 and 2015
tax_burden = tax_burden[(tax_burden['year'] >= 1995) & (tax_burden['year'] <= 2015)]

In [32]:
# Checking which different elements (countries) are in the df1 and tax_burden
list1 = df1['name'].unique()
list2 = tax_burden['name'].unique()

set1 = set(list1)
set2 = set(list2)

equal_elements = set1.intersection(set2)
different_elements = set1.symmetric_difference(set2)

different_elements

{'Afghanistan',
 'Bangladesh',
 'Bangladesh ',
 'Barbados',
 'Belize',
 'Bosnia and Herzegovina',
 'Brunei Darussalam',
 'Burma',
 'Cape Verde',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 'Costa Rica',
 'Costa Rica ',
 "Cote d'Ivoire",
 'Cuba',
 'Czech Republic',
 'Czechia',
 "Côte d'Ivoire ",
 'Democratic Republic of Congo',
 'Djibouti',
 'Dominica',
 'Egypt',
 'Egypt, Arab Rep.',
 'El Salvador',
 'El Salvador ',
 'Eswatini',
 'Gambia, The',
 'Guatemala',
 'Guatemala ',
 'Honduras',
 'Honduras ',
 'Hong Kong',
 'Hungary',
 'Hungary ',
 'Iceland',
 'Iran',
 'Iran, Islamic Rep.',
 'Iraq',
 'Jamaica',
 'Jamaica ',
 'Kiribati',
 'Korea, Rep.',
 'Kosovo',
 'Kyrgyz Republic',
 'Kyrgyz Republic ',
 'Lao PDR',
 'Laos',
 'Liechtenstein',
 'Macau',
 'Macedonia',
 'Malaysia',
 'Malaysia ',
 'Maldives',
 'Malta',
 'Micronesia',
 'Montenegro',
 'Mozambique',
 'Mozambique ',
 'Myanmar',
 'Netherlands',
 'Nicaragua',
 'Nicaragua ',
 'North Korea',
 'Pakistan',
 'Pakistan ',
 'Panama ',
 'Paraguay',
 'Par

Repeting the same process made in the democracy data


In [33]:
# Removing the blank space at the end of the countries' name in each row
tax_burden['name'] = tax_burden['name'].str.rstrip()

In [34]:
tax_burden['name'] = [x.replace('Cape Verde', "Cabo Verde") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Burma', "Myanmar") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Turkey', "Turkiye") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('The Netherlands', "Netherlands") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Democratic Republic of Congo', "Congo, Dem. Rep.") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Republic of Congo', "Congo, Rep.") for x in tax_burden['name']]
tax_burden['name'] = [x.replace("Côte d'Ivoire", "Cote d'Ivoire") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Czech Republic', "Czechia") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Egypt', "Egypt, Arab Rep.") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Swaziland', "Eswatini") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('The Gambia', "Gambia, The") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Iran', "Iran, Islamic Rep.") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('South Korea', "Korea, Rep.") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Laos', "Lao PDR") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('The Philippines', "Philippines") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Russia', "Russian Federation") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Slovakia', "Slovak Republic") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Syria', "Syrian Arab Republic") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Burma', "Myanmar") for x in tax_burden['name']]
tax_burden['name'] = [x.replace("Venezuela", "Venezuela, RB") for x in tax_burden['name']] 
tax_burden['name'] = [x.replace("Yemen", "Yemen, Rep.") for x in tax_burden['name']]

In [35]:
# Countries that are in tax_burden but not in df1
countries_to_drop3 = ['Afghanistan', 'Barbados', 'Belize', 'Bosnia and Herzegovina', 'Brunei Darussalam', 'Cuba', 'Djibouti', 
                      'Dominica', 'Hong Kong', 'Iceland', 'Iraq', 'Kiribati', 'Kosovo', 'Liechtenstein', 'Macau', 'Macedonia', 
                      'Maldives', 'Malta', 'Micronesia', 'Montenegro', 'North Korea', 'Panama', 'Saint Lucia', 
                      'Saint Vincent and the Grenadines', 'Samoa', 'Serbia', 'Seychelles', 'Somalia', 'Sudan', 
                      'São Tomé and Príncipe', 'Taiwan', 'The Bahamas', 'Timor-Leste', 'Tonga', 'Turkmenistan', 
                      'Uzbekistan', 'Vanuatu']

tax_burden = tax_burden[~tax_burden['name'].isin(countries_to_drop3)]

In [36]:
# Merging df1 and tax_burden
df2 = pd.merge(df1, tax_burden, left_on=['name', 'year'], right_on=['name', 'year'])
df2 = df2.rename(columns={'Tax Burden': 'tax_burden'})

### Diversity of exports

In [37]:
# Importing the data
diversity = pd.read_csv('export_diversity.csv')

# Droping some columns
columns_to_drop = ['Country Code', 'Indicator Name', 'Indicator Code', 'Attribute', 'Unnamed: 29']
diversity = diversity.drop(columns=columns_to_drop)

# Renaming the column
diversity = diversity.rename(columns={'Country Name':'name'})

# Reshaping
diversity = pd.melt(diversity, id_vars='name', var_name='year', value_name='diversity')

# Sorting the data
diversity = diversity.sort_values(['name', 'year'], ascending=[True, True])

Repeting the process made in the previous data sets

In [38]:
list1 = diversity['name'].unique()
list2 = df2['name'].unique()

set1 = set(list1)
set2 = set(list2)

equal_elements = set1.intersection(set2)
different_elements = set1.symmetric_difference(set2)

different_elements

{'Afghanistan, Islamic Republic of',
 'Andorra',
 'Anguilla',
 'Antigua and Barbuda',
 'Armenia',
 'Armenia, Republic of',
 'Aruba',
 'Azerbaijan',
 'Azerbaijan, Republic of',
 'Bahamas, The',
 'Bahrain',
 'Bahrain, Kingdom of',
 'Barbados',
 'Belgium-Luxembourg',
 'Belize',
 'Bermuda',
 'Bosnia and Herzegovina',
 'Brunei Darussalam',
 'Cayman Islands',
 'China',
 'China, P.R.: Hong Kong',
 'China, P.R.: Macao',
 'China, P.R.: Mainland',
 'Congo, Dem. Rep.',
 'Congo, Democratic Republic of',
 'Congo, Rep.',
 'Congo, Republic of',
 'Cuba',
 'Czech Republic',
 'Czechia',
 'Djibouti',
 'Dominica',
 'Egypt',
 'Egypt, Arab Rep.',
 'Eswatini',
 'Faroe Islands',
 'French Territories: French Polynesia',
 'French Territories: New Caledonia',
 'Gibraltar',
 'Greenland',
 'Grenada',
 'Iceland',
 'Iran, Islamic Rep.',
 'Iran, Islamic Republic of',
 'Iraq',
 'Kiribati',
 "Korea, Democratic People's Rep. of",
 'Korea, Rep.',
 'Korea, Republic of',
 'Lao PDR',
 "Lao People's Democratic Republic",
 'M

In [39]:
diversity['name'] = [x.replace('Armenia, Republic of', "Armenia") for x in diversity['name']]
diversity['name'] = [x.replace('Azerbaijan, Republic of', "Azerbaijan") for x in diversity['name']]
diversity['name'] = [x.replace('Bahrain, Kingdom of', "Bahrain") for x in diversity['name']]
diversity['name'] = [x.replace('China, P.R.: Mainland', "China") for x in diversity['name']]
diversity['name'] = [x.replace('Congo, Democratic Republic of', "Congo, Dem. Rep.") for x in diversity['name']]
diversity['name'] = [x.replace('Congo, Republic of', "Congo, Rep.") for x in diversity['name']]
diversity['name'] = [x.replace('Czech Republic', "Czechia") for x in diversity['name']]
diversity['name'] = [x.replace('Egypt', "Egypt, Arab Rep.") for x in diversity['name']]
diversity['name'] = [x.replace('Swaziland', "Eswatini") for x in diversity['name']]
diversity['name'] = [x.replace('Iran, Islamic Republic of', "Iran, Islamic Rep.") for x in diversity['name']]
diversity['name'] = [x.replace('Korea, Republic of', "Korea, Rep.") for x in diversity['name']]
diversity['name'] = [x.replace( "Lao People's Democratic Republic", 'Lao PDR') for x in diversity['name']]
diversity['name'] = [x.replace('Turkey', "Turkiye") for x in diversity['name']]
diversity['name'] = [x.replace('Venezuela, Republica Bolivariana de', "Venezuela, RB") for x in diversity['name']]
diversity['name'] = [x.replace('Yemen, Republic of', "Yemen, Rep.") for x in diversity['name']]

In [40]:
countries_to_drop4 = ['Afghanistan, Islamic Republic of', 'Andorra', 'Anguilla', 'Antigua and Barbuda', 'Aruba', 'Bahamas, The',
                      'Barbados', 'Belgium-Luxembourg', 'Belize', 'Bermuda', 'Bosnia and Herzegovina', 'Brunei Darussalam', 
                      'Cayman Islands', 'China, P.R.: Hong Kong', 'China, P.R.: Macao', 'Cuba', 'Djibouti', 'Dominica', 
                      'Faroe Islands', 'French Territories: French Polynesia', 'French Territories: New Caledonia', 'Gibraltar',
                      'Greenland', 'Grenada', 'Iceland', 'Iraq', 'Kiribati', "Korea, Democratic People's Rep. of", 'Maldives', 'Malta', 'Marshall Islands, Republic of', 'Montserrat', 
                      'Netherlands Antilles', 'North Macedonia, Republic of', 'Panama', 'Samoa', 'Sao Tome and Principe', 
                      'Serbia and Montenegro', 'Seychelles', 'Somalia', 'South Sudan', 'St. Kitts and Nevis', 'St. Lucia', 
                      'St. Vincent and the Grenadines', 'Sudan', 'Tonga', 'Turkmenistan', 'Tuvalu', 'Uzbekistan', 'Vanuatu', 
                      'Virgin Islands, British']

diversity = diversity[~diversity['name'].isin(countries_to_drop4)]

In [41]:
# Converting the year column in int64 since it is not in ideal format
diversity['year'] = diversity['year'].astype('int64')

In [42]:
# Merging df2 and diversity
df3 = pd.merge(df2, diversity, left_on=['name', 'year'], right_on=['name', 'year'])
df3 = df3.rename(columns={'diversity':'diversity'})

In [43]:
df3.head()

,name,economy,year,fdi,property_time,cost_procedures,business_time,business_procedure,tribute_time,governement_spending,exchange,inflation,gdp_pc,unemployment,democ,tax_burden,diversity
0,Angola,AGO,1995,8.529489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1825.495149,398.120223,NaN,transition,61.6,6.002369
1,Angola,AGO,1996,2.762770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4800.531644,454.375004,NaN,transition,54.6,6.048760
2,Angola,AGO,1997,5.363360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.453022,516.127849,NaN,autocracy,52.6,5.991146
3,Angola,AGO,1998,17.121191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.359348,423.403332,NaN,autocracy,59.1,5.882152
4,Angola,AGO,1999,40.167251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,557.501113,387.689415,NaN,autocracy,47.9,5.903543


### Quality of exports

In [44]:
# Importing the data
quality = pd.read_excel("quality.xlsx")

# Renaming the country column
quality = quality.rename(columns={'country':'name'})

# Reshaping
quality = pd.melt(quality, id_vars='name', var_name='year', value_name='quality')

In [45]:
# checking the df info
quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3960 entries, 0 to 3959
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     3960 non-null   object 
 1   year     3960 non-null   object 
 2   quality  3853 non-null   float64
dtypes: float64(1), object(2)
memory usage: 92.9+ KB


In [46]:
# Transforming the year column into int64
quality['year'] = quality['year'].astype('int64')

In [47]:
list1 = quality['name'].unique()
list2 = df3['name'].unique()

set1 = set(list1)
set2 = set(list2)

equal_elements = set1.intersection(set2)
different_elements = set1.symmetric_difference(set2)

different_elements

{'Antigua and Barbuda',
 'Armenia',
 'Armenia, Republic of',
 'Azerbaijan',
 'Azerbaijan, Republic of',
 'Bahamas, The',
 'Bahrain',
 'Bahrain, Kingdom of',
 'Barbados',
 'Belize',
 'Bhutan',
 'Bosnia and Herzegovina',
 'China',
 'China, P.R.: Hong Kong',
 'China, P.R.: Mainland',
 'Congo, Dem. Rep.',
 'Congo, Democratic Republic of',
 'Congo, Rep.',
 'Congo, Republic of',
 'Croatia',
 'Czech Republic',
 'Czechia',
 'Djibouti',
 'Dominica',
 'Egypt',
 'Egypt, Arab Rep.',
 'Eritrea',
 'Eswatini',
 'Grenada',
 'Guyana',
 'Iceland',
 'Iran, Islamic Rep.',
 'Iran, Islamic Republic of',
 'Iraq',
 'Korea, Rep.',
 'Korea, Republic of',
 'Lao PDR',
 "Lao People's Democratic Republic",
 'Libya',
 'Maldives',
 'Malta',
 'North Macedonia, Republic of',
 'Panama',
 'Papua New Guinea',
 'Sao Tome and Principe',
 'Seychelles',
 'Solomon Islands',
 'St. Kitts and Nevis',
 'St. Lucia',
 'St. Vincent and the Grenadines',
 'Sudan',
 'Swaziland',
 'Turkey',
 'Turkiye',
 'Turkmenistan',
 'Uzbekistan',
 'V

In [48]:
quality['name'] = [x.replace('Armenia, Republic of', "Armenia") for x in quality['name']]
quality['name'] = [x.replace('Azerbaijan, Republic of', "Azerbaijan") for x in quality['name']]
quality['name'] = [x.replace('Bahrain, Kingdom of', "Bahrain") for x in quality['name']]
quality['name'] = [x.replace('China, P.R.: Mainland', "China") for x in quality['name']]
quality['name'] = [x.replace('Congo, Republic of', "Congo, Rep.") for x in quality['name']]
quality['name'] = [x.replace('Congo, Democratic Republic of', "Congo, Dem. Rep.") for x in quality['name']]
quality['name'] = [x.replace('Czech Republic', "Czechia") for x in quality['name']]
quality['name'] = [x.replace('Egypt', "Egypt, Arab Rep.") for x in quality['name']]
quality['name'] = [x.replace('Swaziland', "Eswatini") for x in quality['name']]
quality['name'] = [x.replace('Iran, Islamic Republic of', "Iran, Islamic Rep.") for x in quality['name']]
quality['name'] = [x.replace('Korea, Republic of', "Korea, Rep.") for x in quality['name']]
quality['name'] = [x.replace("Lao People's Democratic Republic", 'Lao PDR') for x in quality['name']]
quality['name'] = [x.replace('Turkey', "Turkiye") for x in quality['name']]
quality['name'] = [x.replace('Venezuela, Republica Bolivariana de', 'Venezuela, RB') for x in quality['name']]
quality['name'] = [x.replace('Yemen, Republic of', 'Yemen, Rep.') for x in quality['name']]

In [49]:
countries_to_drop5 = ['Antigua and Barbuda', 'Bahamas, The', 'Barbados', 'Belize', 'Bhutan', 'Bosnia and Herzegovina', 
                      'China, P.R.: Hong Kong', 'Croatia', 'Djibouti', 'Dominica', 'Eritrea', 'Grenada', 'Guyana', 'Iceland', 
                      'Iraq', 'Libya', 'Maldives', 'Malta', 'North Macedonia, Republic of', 'Panama', 'Papua New Guinea', 
                      'Sao Tome and Principe', 'Seychelles', 'Solomon Islands', 'St. Kitts and Nevis', 'St. Lucia', 
                      'St. Vincent and the Grenadines', 'Sudan', 'Turkmenistan', 'Uzbekistan']

quality = quality[~quality['name'].isin(countries_to_drop5)]

In [50]:

countries_to_drop6 = ['Bhutan', 'Croatia', 'Eritrea', 'Guyana', 'Libya', 'Papua New Guinea', 'Solomon Islands']
df3 = df3[~df3['name'].isin(countries_to_drop6)]

In [52]:
df4 = pd.merge(df3, quality, left_on=['name', 'year'], right_on=['name', 'year'])
df4 = df4.rename(columns={'quality':'quality'})

In [53]:
df4['name'] = [x.replace('Central African Republic', 'central_african_republic') for x in df4['name']]
df4['name'] = [x.replace('Congo, Dem. Rep.', 'congo_dem_rep') for x in df4['name']]
df4['name'] = [x.replace('Guinea-Bissau', 'guinea_bissau') for x in df4['name']]
df4['name'] = [x.replace('Turkiye', 'turkey') for x in df4['name']]
df4['name'] = [x.replace('Syrian Arab Republic', 'syria') for x in df4['name']]
df4['name'] = [x.replace('Czechia', 'czech_republic') for x in df4['name']]
df4['name'] = [x.replace('Lao PDR', 'laos') for x in df4['name']]

In [54]:
# Selecting only the first name of each country
df4['name'] = df4['name'].str.split(',').str[0].str.strip()

# Only lowercase letters
df4['name'] = df4['name'].str.lower()

# removing accentuation
df4['name'] = df4['name'].apply(lambda x: unidecode(x)) 

# replacing blank space by _
df4['name'] = [x.replace(' ', '_') for x in df4['name']]

### Merging the shadow economy data (target variable) with the other variables

In [55]:
list1 = se_medina['country'].unique()
list2 = df4['name'].unique()

set1 = set(list1)
set2 = set(list2)

equal_elements = set1.intersection(set2)
different_elements = set1.symmetric_difference(set2)

different_elements

{'bahamas',
 'belize',
 'bhutan',
 'bosnia_and_herzegovina',
 'brunei',
 'croatia',
 'eritrea',
 'eswatini',
 'guyana',
 'hong_kong',
 'iceland',
 'libya',
 'maldives',
 'malta',
 'papua_new_guinea',
 'solomon_islands',
 'swaziland',
 'taiwan',
 'vietnam'}

In [56]:
list1 = se_medina['country'].unique()
list2 = df4['name'].unique()

set1 = set(list1)
set2 = set(list2)

equal_elements = set1.intersection(set2)
different_elements = set1.symmetric_difference(set2)

# Exibindo elementos diferentes e seus conjuntos
for element in different_elements:
    if element in set1:
        print(f'{element} is in set 1')
    else:
        print(f'{element} is in set 2')

belize is in set 1
eswatini is in set 2
swaziland is in set 1
malta is in set 1
bosnia_and_herzegovina is in set 1
croatia is in set 1
solomon_islands is in set 1
guyana is in set 1
bahamas is in set 1
iceland is in set 1
eritrea is in set 1
hong_kong is in set 1
papua_new_guinea is in set 1
brunei is in set 1
vietnam is in set 1
maldives is in set 1
taiwan is in set 1
bhutan is in set 1
libya is in set 1


In [57]:
# removing the countries that are in se_medina but not in df4
countries_to_drop7 = ['bahamas','belize','bhutan','bosnia_and_herzegovina','brunei','croatia','eritrea','guyana','hong_kong',
                      'iceland','libya','maldives','malta','papua_new_guinea','solomon_islands','taiwan']

se_medina = se_medina[~se_medina['country'].isin(countries_to_drop7)]

In [58]:
# Renaming the rows that write swaziland by eswatini
se_medina['country'] = [x.replace('swaziland', 'eswatini') for x in se_medina['country']]

# renaming the column
df4 = df4.rename(columns={'name':'country'})

In [59]:
# merging df4 and se_medina
df5 = pd.merge(df4, se_medina, left_on=['country', 'year'], right_on=['country', 'year'])

In [60]:
# Droping the economy column since it's unnecesary
df5 = df5.drop('economy', axis=1)

In [68]:
df5.head()

,country,year,fdi,property_time,cost_procedures,business_time,business_procedure,tribute_time,governement_spending,exchange,inflation,gdp_pc,unemployment,democ,tax_burden,diversity,quality,se_medina
0,angola,1995,8.529489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1825.495149,398.120223,NaN,transition,61.6,6.002369,0.37,52.47
1,angola,1996,2.762770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4800.531644,454.375004,NaN,transition,54.6,6.048760,0.40,46.30
2,angola,1997,5.363360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.453022,516.127849,NaN,autocracy,52.6,5.991146,0.35,50.48
3,angola,1998,17.121191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.359348,423.403332,NaN,autocracy,59.1,5.882152,0.34,49.21
4,angola,1999,40.167251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,557.501113,387.689415,NaN,autocracy,47.9,5.903543,0.34,48.64


In [62]:
print(df5['democ'].value_counts())

democ
democracy          800
full_democracy     647
autocracy          529
open_anocracy      479
closed_anocracy    294
interregnum         31
transition          30
interruption        10
Name: count, dtype: int64


### Creating new variables

In [69]:
# Creating dummies regarding to the democ variables
df6 = pd.get_dummies(df5['democ'], prefix='democ', drop_first='True')
df6 = df6.astype(int)
df6 = pd.concat([df5, df6], axis=1)

In [76]:
# Final df
df6.head()

,fdi,property_time,cost_procedures,business_time,business_procedure,tribute_time,governement_spending,exchange,inflation,unemployment,tax_burden,diversity,quality,se_medina,democ_closed_anocracy,democ_democracy,democ_full_democracy,democ_interregnum,democ_interruption,democ_open_anocracy,democ_transition,lgdp_pc
0,8.529489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1825.495149,NaN,61.6,6.002369,0.37,52.47,0,0,0,0,0,0,1,5.986754
1,2.762770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4800.531644,NaN,54.6,6.048760,0.40,46.30,0,0,0,0,0,0,1,6.118923
2,5.363360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.453022,NaN,52.6,5.991146,0.35,50.48,0,0,0,0,0,0,0,6.246355
3,17.121191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.359348,NaN,59.1,5.882152,0.34,49.21,0,0,0,0,0,0,0,6.048325
4,40.167251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,557.501113,NaN,47.9,5.903543,0.34,48.64,0,0,0,0,0,0,0,5.960205


In [72]:
#  Calculate the natural logarithm of the 'gdp_pc' column
df6['lgdp_pc'] = np.log(df6['gdp_pc'])

In [75]:
# Droping columns that will not be used
df6 = df6.drop(['democ','country', 'year', 'gdp_pc'], axis=1)

### Splitting in train and test datasets

Separating the final data set in train and test. The training data set will be used to train and validate the machine learning algorithms. 

In [306]:
train_set, test_set = train_test_split(df5, test_size=0.2, random_state=0)

### Dealing with missing data

In [307]:
train_set.head()

,country,year,fdi,property_time,cost_procedures,business_time,business_procedure,tribute_time,governement_spending,exchange,inflation,gdp_pc,unemployment,democ,tax_burden,diversity,quality,se_medina
569,colombia,2004,2.660835,23.0,25.6,41.0,12.0,NaN,15.681598,35.863427,7.283033,2811.459451,13.717,democracy,66.7,2.741531,0.81,35.30
402,canada,1997,2.317970,NaN,NaN,NaN,NaN,NaN,19.853346,74.336610,1.144251,21901.562855,9.101,full_democracy,38.1,1.893744,1.00,15.15
2128,poland,2003,2.465699,NaN,21.3,63.0,11.0,NaN,18.921162,69.446215,0.776036,5701.638863,19.370,full_democracy,65.6,1.689366,0.93,26.42
1212,india,2007,2.073394,NaN,NaN,NaN,NaN,NaN,9.862116,45.686269,6.944418,1022.732467,NaN,democracy,76.0,1.813474,0.83,21.03
452,chile,2007,7.811859,31.0,11.1,40.0,9.0,316.0,10.545955,76.977025,5.088757,10456.832384,8.427,full_democracy,78.8,3.734244,0.76,15.10


In [308]:
# Missing data by column (%)
train_set.isnull().mean().sort_values(ascending=False)*100

tribute_time            53.945035
property_time           49.911348
cost_procedures         45.744681
business_time           45.744681
business_procedure      45.744681
unemployment            34.574468
governement_spending     8.643617
exchange                 6.693262
tax_burden               4.787234
quality                  1.152482
inflation                0.620567
fdi                      0.576241
gdp_pc                   0.132979
diversity                0.000000
country                  0.000000
democ                    0.000000
year                     0.000000
se_medina                0.000000
dtype: float64

There is a lot of missing data, especially in relation to the tribute_time, however, if we limit the year range from 2004 to 2014, the proportion of missing data will decrease

In [309]:
train_set = train_set[(train_set['year']>=2004) & (train_set['year'] <= 2014)]

In [310]:
# porportion of missing data after removing the 2015 year
train_set.isnull().mean().sort_values(ascending=False)*100

unemployment            29.797571
tribute_time            15.870445
property_time            8.502024
cost_procedures          8.502024
business_time            8.502024
business_procedure       8.502024
governement_spending     6.396761
exchange                 5.182186
tax_burden               1.214575
fdi                      0.242915
democ                    0.000000
quality                  0.000000
diversity                0.000000
country                  0.000000
gdp_pc                   0.000000
inflation                0.000000
year                     0.000000
se_medina                0.000000
dtype: float64

In [311]:
# Proportion of missing data by country in the column 'unemployment' since it's the variable with the biggest proportion of missing data
missing_by_country = train_set.groupby('country')['unemployment'].apply(lambda x: x.isnull().sum() / x.count()).sort_values(ascending=False)
#print(missing_by_country)

In [312]:
# Collecting the names of countries that have a proportion of missing data in the unemployment column above 5%
countries = missing_by_country[missing_by_country > 5].index.tolist()
#countries

# Removing these countries
train_set = train_set[~train_set['country'].isin(countries)]

In [313]:
len(train_set['country'].unique())

117

In [314]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1039 entries, 569 to 2732
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   country               1039 non-null   object 
 1   year                  1039 non-null   object 
 2   fdi                   1036 non-null   float64
 3   property_time         954 non-null    float64
 4   cost_procedures       954 non-null    float64
 5   business_time         954 non-null    float64
 6   business_procedure    954 non-null    float64
 7   tribute_time          873 non-null    float64
 8   governement_spending  991 non-null    float64
 9   exchange              1003 non-null   float64
 10  inflation             1039 non-null   float64
 11  gdp_pc                1039 non-null   float64
 12  unemployment          854 non-null    float64
 13  democ                 1039 non-null   object 
 14  tax_burden            1033 non-null   float64
 15  diversity             10

In [315]:
# Replacing missing values ​​with the average for each country for each attribute
#mean_by_country = train_set.groupby('country').transform('mean')
#train_set.fillna(mean_by_country, inplace=True)

In [316]:
# Replacing missing values ​​with the average for each country for each attribute - Pnly numeric variables
numeric_columns = train_set.select_dtypes(include='number').columns
mean_by_country = train_set.groupby('country')[numeric_columns].transform('mean')
train_set[numeric_columns] = train_set[numeric_columns].fillna(mean_by_country)

In [317]:
train_set.head(11)

,country,year,fdi,property_time,cost_procedures,business_time,business_procedure,tribute_time,governement_spending,exchange,inflation,gdp_pc,unemployment,democ,tax_burden,diversity,quality,se_medina
569,colombia,2004,2.660835,23.00,25.6,41.000000,12.000000,253.875,15.681598,35.863427,7.283033,2811.459451,13.7170,democracy,66.7000,2.741531,0.81,35.30
452,chile,2007,7.811859,31.00,11.1,40.000000,9.000000,316.000,10.545955,76.977025,5.088757,10456.832384,8.4270,full_democracy,78.8000,3.734244,0.76,15.10
674,czech_republic,2009,2.541341,79.00,9.2,32.000000,9.000000,442.000,20.671443,112.797245,2.587821,19861.697430,6.6620,democracy,80.1631,1.892051,0.98,13.52
2549,tajikistan,2004,13.102214,37.00,43.1,45.285714,10.142857,296.000,11.784736,128.338171,17.431579,305.266044,2.0000,closed_anocracy,85.0000,4.306413,0.60,43.52
438,switzerland,2013,-3.529486,16.00,2.3,18.000000,6.000000,63.000,11.412579,133.693708,-0.086843,87304.330581,4.7470,full_democracy,68.1000,2.454753,1.04,6.56
2673,ukraine,2008,5.688149,113.00,6.8,28.000000,11.000000,860.000,16.973534,94.171892,29.020114,4066.531738,6.3630,democracy,78.9684,2.010335,0.86,36.65
1190,indonesia,2005,2.916115,27.64,22.0,76.900000,12.000000,259.000,8.109508,63.987936,14.331787,1249.397694,7.9450,democracy,77.5000,2.150141,0.79,24.82
330,bolivia,2005,-2.498868,91.00,153.2,49.000000,14.000000,1080.000,15.974558,67.641941,5.940798,1018.313725,5.4500,democracy,88.9000,3.604216,0.73,65.64
1397,kenya,2012,2.447259,72.00,47.5,34.000000,13.000000,352.000,12.271377,51.623692,9.522972,1289.780791,9.7785,democracy,77.7000,2.808761,0.62,30.11
1694,madagascar,2009,13.448543,100.00,46.2,12.000000,9.000000,201.000,12.508634,62.411698,6.929159,455.407374,3.4500,open_anocracy,80.8551,3.712429,0.60,43.33


In [318]:
train_set.isnull().mean().sort_values(ascending=False)*100

governement_spending    3.079885
exchange                2.309913
inflation               0.000000
quality                 0.000000
diversity               0.000000
tax_burden              0.000000
democ                   0.000000
unemployment            0.000000
gdp_pc                  0.000000
country                 0.000000
year                    0.000000
tribute_time            0.000000
business_procedure      0.000000
business_time           0.000000
cost_procedures         0.000000
property_time           0.000000
fdi                     0.000000
se_medina               0.000000
dtype: float64

After the imputation, 'government spending' and 'exchange' still having missing values, this happens because some countries have the 'inf' as observations and not a usual 

In [319]:
# This is seen with the follow code for governement_spending and after for exchange (just change governement_spending by exchange to see all the countries)
missing_by_country = train_set.groupby('country')['governement_spending'].apply(lambda x: x.isnull().sum() / x.count()).sort_values(ascending=False)
#print(missing_by_country)

In [320]:
# Droping these countries
countries_to_drop8 = ['malawi', 'trinidad_and_tobago', 'liberia', 'yemen', "cote_d'ivoire", "nigeria"]
train_set = train_set[~train_set['country'].isin(countries_to_drop8)]

In [321]:
# No more missing values
train_set.isnull().mean().sort_values(ascending=False)

country                 0.0
year                    0.0
quality                 0.0
diversity               0.0
tax_burden              0.0
democ                   0.0
unemployment            0.0
gdp_pc                  0.0
inflation               0.0
exchange                0.0
governement_spending    0.0
tribute_time            0.0
business_procedure      0.0
business_time           0.0
cost_procedures         0.0
property_time           0.0
fdi                     0.0
se_medina               0.0
dtype: float64

### blvGEBFLIWEGVF

In [330]:

# Renaming the columns for better visualization
rename_dictionary = {'se_medina':'Shadow Economy',
                        'lgdp_pc': 'Log. GDP Per Capita', 
                         'democ_transition': 'Dem. Transition', 
                         #'democ_interruption': 'Dem. Interruption',
                        'interruption': 'Dem. Interruption',
                        'democ_interregnum':'Dem. Interregnum',
                        'democ_full_democracy': 'Dem. Full',
                        'democ_democracy':'Democracy',
                        'democ_autocracy':'Autocracy',
                        'democ_closed_anocracy':'Closed Anocracy',
                        'democ_open_anocracy': 'Open Anocracy',
                        'quality':'Quality',
                        'diversity':'Diversity',
                        'tax_burden':'Tax Burden',
                        'unemployment':'Unemployment',
                        'inflation':'Inflation',
                        'exchange':'Exchange',
                        'governement_spending':'Government Spending',
                         'tribute_time':'Tribute Time',
                        'business_procedure':'Business Procedure',
                        'business_time':'Business Time',
                        'cost_procedures':'Cost Procedures',
                        'property_time':'Property Time',
                        'fdi':'FDI'}

train_set_1 = train_set_1.rename(columns=rename_dictionary)
train_set_1.head()


,FDI,Property Time,Cost Procedures,Business Time,Business Procedure,Tribute Time,Government Spending,Exchange,Inflation,Unemployment,democ,Tax Burden,Diversity,Quality,Shadow Economy,Closed Anocracy,Democracy,Dem. Full,Dem. Interregnum,Open Anocracy,Dem. Transition,Log. GDP Per Capita
569,2.660835,23.0,25.6,41.000000,12.000000,253.875,15.681598,35.863427,7.283033,13.717,democracy,66.7000,2.741531,0.81,35.30,0,1,0,0,0,0,7.941459
452,7.811859,31.0,11.1,40.000000,9.000000,316.000,10.545955,76.977025,5.088757,8.427,full_democracy,78.8000,3.734244,0.76,15.10,0,0,1,0,0,0,9.255011
674,2.541341,79.0,9.2,32.000000,9.000000,442.000,20.671443,112.797245,2.587821,6.662,democracy,80.1631,1.892051,0.98,13.52,0,1,0,0,0,0,9.896548
2549,13.102214,37.0,43.1,45.285714,10.142857,296.000,11.784736,128.338171,17.431579,2.000,closed_anocracy,85.0000,4.306413,0.60,43.52,1,0,0,0,0,0,5.721184
438,-3.529486,16.0,2.3,18.000000,6.000000,63.000,11.412579,133.693708,-0.086843,4.747,full_democracy,68.1000,2.454753,1.04,6.56,0,0,1,0,0,0,11.377155


In [331]:
# droping the column 'democ'
train_set_1 = train_set_1.drop('democ', axis=1)

In [332]:
# Changing the order
new_order2 = ['Shadow Economy', 'FDI', 'Property Time', 'Cost Procedures', 'Business Time', 'Business Procedure',
               'Tribute Time', 'Government Spending', 'Exchange', 'Inflation', 'Unemployment', 'Tax Burden', 
               'Diversity', 'Quality', 'Closed Anocracy', 'Democracy', 'Dem. Full', 'Dem. Interregnum', 
               'Dem. Interruption', 'Open Anocracy', 'Dem. Transition', 'Log. GDP Per Capita']

train_set_1 = train_set_1.reindex(columns=new_order2)

In [333]:
train_set_1.head()

,Shadow Economy,FDI,Property Time,Cost Procedures,Business Time,Business Procedure,Tribute Time,Government Spending,Exchange,Inflation,Unemployment,Tax Burden,Diversity,Quality,Closed Anocracy,Democracy,Dem. Full,Dem. Interregnum,Dem. Interruption,Open Anocracy,Dem. Transition,Log. GDP Per Capita
569,35.30,2.660835,23.0,25.6,41.000000,12.000000,253.875,15.681598,35.863427,7.283033,13.717,66.7000,2.741531,0.81,0,1,0,0,NaN,0,0,7.941459
452,15.10,7.811859,31.0,11.1,40.000000,9.000000,316.000,10.545955,76.977025,5.088757,8.427,78.8000,3.734244,0.76,0,0,1,0,NaN,0,0,9.255011
674,13.52,2.541341,79.0,9.2,32.000000,9.000000,442.000,20.671443,112.797245,2.587821,6.662,80.1631,1.892051,0.98,0,1,0,0,NaN,0,0,9.896548
2549,43.52,13.102214,37.0,43.1,45.285714,10.142857,296.000,11.784736,128.338171,17.431579,2.000,85.0000,4.306413,0.60,1,0,0,0,NaN,0,0,5.721184
438,6.56,-3.529486,16.0,2.3,18.000000,6.000000,63.000,11.412579,133.693708,-0.086843,4.747,68.1000,2.454753,1.04,0,0,1,0,NaN,0,0,11.377155


### Data Padronization

In [339]:
scaler = MinMaxScaler()
final_train_df = scaler.fit_transform(train_set_1.values)
final_train_df = pd.DataFrame(final_train_df, columns=train_set_1.columns)

In [340]:
final_train_df.head()

,Shadow Economy,FDI,Property Time,Cost Procedures,Business Time,Business Procedure,Tribute Time,Government Spending,Exchange,Inflation,Unemployment,Tax Burden,Diversity,Quality,Closed Anocracy,Democracy,Dem. Full,Dem. Interregnum,Dem. Interruption,Open Anocracy,Dem. Transition,Log. GDP Per Capita
0,0.463128,0.178256,0.031930,0.017163,0.058738,0.611111,0.088175,0.309713,0.034116,0.263328,0.383352,0.510727,0.269548,0.662791,0.0,1.0,0.0,0.0,NaN,0.0,0.0,0.435809
1,0.142085,0.193510,0.043541,0.007442,0.057288,0.444444,0.112320,0.182196,0.136070,0.244986,0.233324,0.688819,0.472438,0.604651,0.0,0.0,1.0,0.0,NaN,0.0,0.0,0.631659
2,0.116974,0.177902,0.113208,0.006168,0.045685,0.444444,0.161290,0.433610,0.224897,0.224080,0.183267,0.708881,0.095931,0.860465,0.0,1.0,0.0,0.0,NaN,0.0,0.0,0.727312
3,0.593770,0.209177,0.052250,0.028895,0.064954,0.507937,0.104547,0.212955,0.263435,0.348164,0.051049,0.780073,0.589378,0.418605,1.0,0.0,0.0,0.0,NaN,0.0,0.0,0.104766
4,0.006357,0.159924,0.021771,0.001542,0.025381,0.277778,0.013991,0.203714,0.276716,0.201721,0.128956,0.531332,0.210936,0.930233,0.0,0.0,1.0,0.0,NaN,0.0,0.0,0.948070


### Spliting the data between train and validation

In [341]:
# separating the data into train and validation
X_train, X_valid, y_train, y_valid = train_test_split(
    final_train_df.drop(labels=['Shadow Economy'], axis=1),
    final_train_df['Shadow Economy'],
    test_size=0.2,
    random_state=0
)

### Regressions

### Linear Regression

In [342]:
from sklearn.linear_model import LinearRegression

# Create a linear regression model
model = LinearRegression()

# Train the model on the training set
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_valid)

# Evaluate the model using MAPE
mape = np.mean(np.abs((y_valid - y_pred) / y_valid)) * 100
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [343]:
X_train.isnull().sum()

FDI                      0
Property Time            0
Cost Procedures          0
Business Time            0
Business Procedure       0
Tribute Time             0
Government Spending      0
Exchange                 0
Inflation                0
Unemployment             0
Tax Burden               0
Diversity                0
Quality                  0
Closed Anocracy          0
Democracy                0
Dem. Full                0
Dem. Interregnum         0
Dem. Interruption      797
Open Anocracy            0
Dem. Transition          0
Log. GDP Per Capita      0
dtype: int64